In [1]:
from yahooquery import Ticker
import yfinance as yf
import pandas as pd
from datetime import datetime, date, timedelta
import warnings
from collections import defaultdict
import time


warnings.filterwarnings("ignore", category=FutureWarning, module="yahooquery")

def get_raw_yahoo(api, symbol_list, start_date):
    try:
        # To avoid time zone issue, end_date is set to the date of tomorrow
        end_date = date.today() + timedelta(days=1)
        
        if api=="yahooquery":                 
            # Attempt to fetch data from Yahoo Finance
            data = Ticker(symbol_list)            
            # Fetch historical data
            hist_data = data.history(start=start_date, end=end_date, interval="1d")
            hist_data = hist_data.reset_index()
            # print(hist_data)
        else:   
            # Fetch historical data
            hist_data = yf.download(
                tickers=symbol_list,
                start=start_date,
                end=end_date,
                interval="1d",
                group_by="ticker",  # Keep data grouped by ticker if multiple symbol_list are provided
                auto_adjust=False,  # Keep original values without adjustment
            )
            
            # Check if the data needs reshaping for multiple tickers
            hist_data = hist_data.stack(level=0, future_stack=True).reset_index()    
            hist_data.rename(columns={"Date": "date", "Ticker":"symbol", "Open": "open", "High": "high", 
                                      "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)
        
        # print(hist_data)
        # # Select only required columns
        columns_to_select = ["date", "symbol", "open", "high", "low", "close", "volume"]
        hist_data = hist_data[columns_to_select]
        # date could be 2012-1-1 or 2012-1-1 12:33:55.000 +5:00:00. Such mixed formats will cause problems
        # The following is to convert date to string then only get the left 10 chars for date
        hist_data["date"] = hist_data["date"].astype(str).str.slice(0, 10)
        hist_data["date"] = pd.to_datetime(hist_data["date"])
        
        # Add a new field for the current datetime
        # hist_data["import_time"] = pd.to_datetime(import_time).tz_localize(None)

        # Return the processed data
        return hist_data
    
    except Exception as e:
        # Handle any exception that occurred during the execution
        print(f"An error occurred: {e}, {symbol_list}, {start_date}")
        
        # Return an empty DataFrame with the same columns as the expected result
        return pd.DataFrame(columns=["date", "symbol", "open", "high", "low", "close", "volume"]) 

In [13]:
from yahooquery import Ticker

# List of symbols
symbol_list = ['TSLA','C','NVDA']

# Create a Ticker object
data = Ticker(symbol_list)

# Fetch key statistics for the symbols
key_stats = data.key_stats

print(key_stats)
# Extract P/E ratio for each symbol
for symbol in symbol_list:
    pe_ratio = key_stats[symbol].get('forwardPE', 'N/A')  # 'trailingPE' is the P/E ratio
    print(f"{symbol}: P/E Ratio = {pe_ratio}")




{'TSLA': {'maxAge': 1, 'priceHint': 2, 'enterpriseValue': 1228948242432, 'forwardPE': 103.642075, 'profitMargins': 0.13075, 'floatShares': 2793105010, 'sharesOutstanding': 3216519936, 'sharesShort': 63254084, 'sharesShortPriorMonth': 80765425, 'sharesShortPreviousMonthDate': '2024-12-12 18:00:00', 'dateShortInterest': '2025-01-14 18:00:00', 'sharesPercentSharesOut': 0.0197, 'heldPercentInsiders': 0.1291, 'heldPercentInstitutions': 0.47744998, 'shortRatio': 0.7, 'shortPercentOfFloat': 0.0226, 'beta': 2.295, 'impliedSharesOutstanding': 3210060032, 'category': None, 'bookValue': 21.806, 'priceToBook': 18.554527, 'fundFamily': None, 'legalType': None, 'lastFiscalYearEnd': '2023-12-30 18:00:00', 'nextFiscalYearEnd': '2024-12-30 18:00:00', 'mostRecentQuarter': '2024-09-29 19:00:00', 'earningsQuarterlyGrowth': 0.169, 'netIncomeToCommon': 12743000064, 'trailingEps': 2.06, 'forwardEps': 3.24, 'lastSplitFactor': '3:1', 'lastSplitDate': '2022-08-24 19:00:00', 'enterpriseToRevenue': 12.65, 'enterp

In [16]:
from yahooquery import Ticker

# List of symbols
symbol_list = ['AAPL', 'MSFT', 'DBMF', 'DGS']

# Create a Ticker object
data = Ticker(symbol_list)

# Fetch asset profile for the symbols
asset_profile = data.asset_profile
# print(asset_profile)
# Extract industry and sector for each symbol
for symbol in symbol_list:
    profile = asset_profile.get(symbol, {})
    industry = profile.get('industry', 'N/A')
    sector = profile.get('sector', 'N/A')
    print(f"{symbol}: Industry = {industry}, Sector = {sector}")

AAPL: Industry = Consumer Electronics, Sector = Technology
MSFT: Industry = Software - Infrastructure, Sector = Technology
DBMF: Industry = N/A, Sector = N/A
DGS: Industry = N/A, Sector = N/A


In [47]:
from collections import defaultdict
import time
import random

def get_raw_yahoo_by_looping_groups(api, group_date_symbol_list):

    largest_group_id = max(group_date_symbol_list, key=lambda x: x[0])[0]
    # Initialize a defaultdict to store the symbol_list for each (group_date, group_id)
    grouped_symbol_list = defaultdict(list)
    
    # Iterate over the data to group symbol_list by (group_date, group_id)
    for group_id, group_start_date, symbol in group_date_symbol_list:
        # Use a tuple of (group_date, group_id) as the key and append the symbol to the list
        grouped_symbol_list[(group_id, group_start_date)].append(symbol)
    
    warnings.filterwarnings("ignore", category=FutureWarning, module="yahooquery")
    
    stacked_hist_panda_df=[]
    for group, group_symbol_list in grouped_symbol_list.items():
        group_id, group_start_date = group
        print(f"Working on {group_id}/{largest_group_id}")
        # print(f"Group Date: {group_start_date}, Group Number: {group_id}, symbol_list: {group_symbol_list}")
        hist_group_panda_df=get_raw_yahoo(api, group_symbol_list, group_start_date)    
        print(f"{len(hist_group_panda_df)} records in {group_id}/{largest_group_id}.")
        
        stacked_hist_panda_df.append(hist_group_panda_df)        
        sleep_time = random.randint(1, 5)
        print(f"Sleeping for {sleep_time} seconds...")
        time.sleep(sleep_time) 
        
    consolidated_hist_panda_df = pd.concat(stacked_hist_panda_df, ignore_index=True)
    
    return consolidated_hist_panda_df

In [48]:
from yahooquery import Ticker
import yfinance as yf
import pandas as pd
from datetime import datetime, date, timedelta
import warnings
from collections import defaultdict
import time


warnings.filterwarnings("ignore", category=FutureWarning, module="yahooquery")

def get_raw_yahoo(api, symbol_list, start_date):
    try:
        # To avoid time zone issue, end_date is set to the date of tomorrow
        end_date = date.today() + timedelta(days=1)
        
        if api=="yahooquery":                 
            # Attempt to fetch data from Yahoo Finance
            data = Ticker(symbol_list)            
            # Fetch historical data
            hist_data = data.history(start=start_date, end=end_date, interval="1d")
            hist_data = hist_data.reset_index()
            # print(hist_data)
        else:   
            # Fetch historical data
            hist_data = yf.download(
                tickers=symbol_list,
                start=start_date,
                end=end_date,
                interval="1d",
                group_by="ticker",  # Keep data grouped by ticker if multiple symbol_list are provided
                auto_adjust=False,  # Keep original values without adjustment
            )
            
            # Check if the data needs reshaping for multiple tickers
            hist_data = hist_data.stack(level=0, future_stack=True).reset_index()    
            hist_data.rename(columns={"Date": "date", "Ticker":"symbol", "Open": "open", "High": "high", 
                                      "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)
        
        # print(hist_data)
        # # Select only required columns
        columns_to_select = ["date", "symbol", "open", "high", "low", "close", "volume"]
        hist_data = hist_data[columns_to_select]
        # date could be 2012-1-1 or 2012-1-1 12:33:55.000 +5:00:00. Such mixed formats will cause problems
        # The following is to convert date to string then only get the left 10 chars for date
        hist_data["date"] = hist_data["date"].astype(str).str.slice(0, 10)
        hist_data["date"] = pd.to_datetime(hist_data["date"])
        
        # Add a new field for the current datetime
        # hist_data["import_time"] = pd.to_datetime(import_time).tz_localize(None)

        # Return the processed data
        return hist_data
    
    except Exception as e:
        # Handle any exception that occurred during the execution
        print(f"An error occurred: {e}")
        
        # Return an empty DataFrame with the same columns as the expected result
        return pd.DataFrame(columns=["date", "symbol", "open", "high", "low", "close", "volume"])

In [82]:

def get_market_quote(symbol_list: list) -> pd.DataFrame:
    """
    Fetch instant market data for a list of symbols.

    Args:
        symbol_list (list): List of stock symbols.

    Returns:
        pd.DataFrame: DataFrame containing instant market data.
    """
    try:
        # Fetch data using yahooquery
        t = Ticker(symbol_list)
        prices = t.price  # Access price data

        # Extract data into a dictionary
        market_data = {
            symbol: {
                "exchange_name": details.get("exchangeName", "NULL"),
                "current_market_state": details.get("marketState", "NULL"),
                "pre_market_time": details.get("preMarketTime", "NULL"),
                "pre_market_price": details.get("preMarketPrice", "NULL"),
                "pre_market_change": details.get("preMarketChange", "NULL"),
                "pre_market_change_percent": details.get("preMarketChangePercent", "NULL"),
                "regular_market_time": details.get("regularMarketTime", "NULL"),
                "regular_market_price": details.get("regularMarketPrice", "NULL"),
                "regular_market_change": details.get("regularMarketChange", "NULL"),
                "regular_market_change_percent": details.get("regularMarketChangePercent", "NULL"),
                "post_market_time": details.get("postMarketTime", "NULL"),
                "post_market_price": details.get("postMarketPrice", "NULL"),
                "post_market_change": details.get("postMarketChange", "NULL"),
                "post_market_change_percent": details.get("postMarketChangePercent", "NULL"),
            }
            for symbol, details in prices.items()
        }

        # Convert the dictionary to a Pandas DataFrame
        market_panda_df = pd.DataFrame.from_dict(market_data, orient="index")
        market_panda_df.index.name = "symbol"  # Set the index name to "symbol"
        market_panda_df = market_panda_df.reset_index()
        return market_panda_df

    except Exception as e:
        # Handle exceptions and print error message
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return pd.DataFrame()  # Return an empty DataFrame on error

# symbol_list=["C","AAXJ"]
# get_market_records(symbol_list)

In [83]:
def get_market_quote_consolidated(group_symbol_list: list) -> pd.DataFrame:
    """
    Fetch and consolidate EOD records for grouped symbols.

    Args:
        group_date_symbol_list (list): List of tuples containing (group_id, group_start_date, symbol).

    Returns:
        pd.DataFrame: Consolidated DataFrame of EOD records.
    """
    try:
        largest_group_id = max(group_symbol_list, key=lambda x: x[0])[0]
        grouped_symbols = defaultdict(list)
        
        # Group symbols by (group_id, group_start_date)
        for group_id, symbol in group_symbol_list:
            grouped_symbols[(group_id)].append(symbol)
        
        stacked_group_panda_dfs = []
        for group_id, group_symbols in grouped_symbols.items():
            print(f"Processing group {group_id}/{largest_group_id} with {len(group_symbols)} symbols...")
            group_panda_df = get_market_quote(group_symbols)
            print(f"Retrieved {len(group_panda_df)} records for group {group_id}/{largest_group_id}.")
            
            stacked_group_panda_dfs.append(group_panda_df)
            sleep_time = random.randint(1, 5)
            print(f"Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
        
        # Combine all DataFrames into one
        consolidated_market_panda_df = pd.concat(stacked_group_panda_dfs, ignore_index=True)
        return consolidated_market_panda_df
    
    except Exception as e:
        # Handle exceptions and return an empty DataFrame
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return pd.DataFrame(columns=["symbol", "exchange_name", "current_market_state", "pre_market_time", "pre_market_price", "pre_market_change", \
                                     "pre_market_change_percent", "regular_market_time", "regular_market_price", \
                                     "regular_market_change", "regular_market_change_percent", "post_market_time", \
                                     "post_market_price", "post_market_change", "post_market_change_percent"])



In [85]:
# group_symbol_list=[(1,'C'), (1,'AAXJ'), (2,'ACWI'), (2,'ACWV')]
# get_market_quote_consolidated(group_symbol_list)

Processing group 1/2 with 2 symbols...
Retrieved 2 records for group 1/2.
Sleeping for 4 seconds...
Processing group 2/2 with 2 symbols...
Retrieved 2 records for group 2/2.
Sleeping for 3 seconds...


,symbol,exchange_name,current_market_state,pre_market_time,pre_market_price,pre_market_change,pre_market_change_percent,regular_market_time,regular_market_price,regular_market_change,regular_market_change_percent,post_market_time,post_market_price,post_market_change,post_market_change_percent
0,C,NYSE,POSTPOST,2025-01-29 08:29:59,79.78,-0.160004,-0.002002,2025-01-29 15:00:01,80.63,0.689995,0.008631,2025-01-29 18:59:21,81.1,0.470001,0.005829
1,AAXJ,NasdaqGM,POSTPOST,NULL,NULL,NULL,NULL,2025-01-29 15:00:01,72.23,0.020004,0.000277,NULL,NULL,NULL,NULL
2,ACWI,NasdaqGM,POSTPOST,2025-01-29 07:55:53,121.47,0.07,0.000577,2025-01-29 15:00:02,121.04,-0.360001,-0.002965,2025-01-29 16:00:04,122.8,1.76,0.014541
3,ACWV,Cboe US,POSTPOST,NULL,NULL,NULL,NULL,2025-01-29 15:00:00,111.62,0.060005,0.000538,2025-01-29 15:07:12,108.14,-3.49,-0.031264


In [89]:
x=[(1,'2004-01-01'), (1,'AAXJ'), (2,'ACWI'), (2,'ACWV')]
for i in range(len(x[0])):
    print(type(x[0][i]))


<class 'int'>
<class 'str'>


In [21]:
import pandas as pd
from yahooquery import Ticker
import traceback

def get_combined_data(symbol_list: list) -> pd.DataFrame:
    """
    Fetch both asset profile and key statistics data for a list of symbols,
    and combine them into a single record per symbol.

    Args:
        symbol_list (list): List of stock symbols.

    Returns:
        pd.DataFrame: DataFrame containing combined asset profile and key statistics data.
    """
    try:
        # Fetch data using yahooquery
        t = Ticker(symbol_list)

        # Fetch asset profile
        asset_profile = t.asset_profile

        # Fetch key statistics
        key_stats = t.key_stats

        # Combine data into a single dictionary
        combined_data = {}
        for symbol in symbol_list:
            # Get asset profile data for the symbol
            profile_data = asset_profile.get(symbol, {})
            # Get key statistics data for the symbol
            stats_data = key_stats.get(symbol, {})

            # Combine both datasets into a single record
            combined_data[symbol] = {
                # Asset Profile Fields
                
                "industry": profile_data.get("industry", "NULL"),
                "sector": profile_data.get("sector", "NULL"),

                # Key Statistics Fields
                "enterprise_value": stats_data.get("enterpriseValue", "NULL"),
                "forward_pe": stats_data.get("forwardPE", "NULL"),
                "profit_margins": stats_data.get("profitMargins", "NULL"),
                "float_shares": stats_data.get("floatShares", "NULL"),
                "shares_outstanding": stats_data.get("sharesOutstanding", "NULL"),
                "shares_short": stats_data.get("sharesShort", "NULL"),
                "shares_short_prior_month": stats_data.get("sharesShortPriorMonth", "NULL"),
                "short_ratio": stats_data.get("shortRatio", "NULL"),
                "short_percent_of_float": stats_data.get("shortPercentOfFloat", "NULL"),
                "beta": stats_data.get("beta", "NULL"),
                "book_value": stats_data.get("bookValue", "NULL"),
                "price_to_book": stats_data.get("priceToBook", "NULL"),
                "last_fiscal_year_end": stats_data.get("lastFiscalYearEnd", "NULL"),
                "next_fiscal_year_end": stats_data.get("nextFiscalYearEnd", "NULL"),
                "most_recent_quarter": stats_data.get("mostRecentQuarter", "NULL"),
                "earnings_quarterly_growth": stats_data.get("earningsQuarterlyGrowth", "NULL"),
                "net_income_to_common": stats_data.get("netIncomeToCommon", "NULL"),
                "trailing_eps": stats_data.get("trailingEps", "NULL"),
                "forward_eps": stats_data.get("forwardEps", "NULL"),
                "peg_ratio": stats_data.get("pegRatio", "NULL"),
                "last_split_factor": stats_data.get("lastSplitFactor", "NULL"),
                "last_split_date": stats_data.get("lastSplitDate", "NULL"),
            }

        # Convert the dictionary to a Pandas DataFrame
        combined_df = pd.DataFrame.from_dict(combined_data, orient="index")
        combined_df.index.name = "symbol"  # Set the index name to "symbol"
        combined_df = combined_df.reset_index()

        # Extract column names as a comma-delimited string
        column_names = ",".join(combined_df.columns)
        print("Column names:", column_names)

        return combined_df

    except Exception as e:
        # Handle exceptions and print error message
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return pd.DataFrame()  # Return an empty DataFrame on error

# Example usage
symbol_list = ['SPXL', 'C', 'NVDA']
combined_df = get_combined_data(symbol_list)
print(combined_df)

Column names: symbol,industry,sector,enterprise_value,forward_pe,profit_margins,float_shares,shares_outstanding,shares_short,shares_short_prior_month,short_ratio,short_percent_of_float,beta,book_value,price_to_book,last_fiscal_year_end,next_fiscal_year_end,most_recent_quarter,earnings_quarterly_growth,net_income_to_common,trailing_eps,forward_eps,peg_ratio,last_split_factor,last_split_date
  symbol             industry              sector enterprise_value forward_pe  \
0   SPXL                 NULL                NULL             NULL       NULL   
1      C  Banks - Diversified  Financial Services      23297296384   8.804748   
2   NVDA       Semiconductors          Technology    3553896693760  27.001379   

  profit_margins float_shares shares_outstanding shares_short  \
0           NULL         NULL               NULL         NULL   
1        0.11524   1829458269         1877100032     36434036   
2        0.55041  23541441000        24490000384    246400791   

  shares_short_prior_

In [22]:
import pandas as pd
from yahooquery import Ticker
import traceback

def get_etf_profile(symbol_list: list) -> pd.DataFrame:
    """
    Fetch ETF profile data for a list of symbols.

    Args:
        symbol_list (list): List of ETF symbols.

    Returns:
        pd.DataFrame: DataFrame containing ETF profile data.
    """
    try:
        # Fetch ETF profile data using yahooquery
        t = Ticker(symbol_list)
        etf_profile = t.fund_profile  # Access ETF profile data

        # Extract data into a dictionary
        profile_data = {
            symbol: {
                "category": details.get("category", "NULL"),
                "family": details.get("family", "NULL"),
                "management_info": details.get("managementInfo", "NULL"),
                "investment_strategy": details.get("investmentStrategy", "NULL"),
                "sector_weightings": details.get("sectorWeightings", "NULL"),
                "top_holdings": details.get("topHoldings", "NULL"),
                "total_assets": details.get("totalAssets", "NULL"),
                "ytd_return": details.get("ytdReturn", "NULL"),
                "beta_3y": details.get("beta3Year", "NULL"),
                "expense_ratio": details.get("expenseRatio", "NULL"),
                "inception_date": details.get("inceptionDate", "NULL"),
            }
            for symbol, details in etf_profile.items()
        }

        # Convert the dictionary to a Pandas DataFrame
        profile_df = pd.DataFrame.from_dict(profile_data, orient="index")
        profile_df.index.name = "symbol"  # Set the index name to "symbol"
        profile_df = profile_df.reset_index()

        # Extract column names as a comma-delimited string
        column_names = ",".join(profile_df.columns)
        print("Column names:", column_names)

        return profile_df

    except Exception as e:
        # Handle exceptions and print error message
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return pd.DataFrame()  # Return an empty DataFrame on error

# Example usage
etf_symbol_list = ['SPY', 'QQQ', 'VOO']  # Example ETF symbols
etf_profile_df = get_etf_profile(etf_symbol_list)
print(etf_profile_df)

Column names: symbol,category,family,management_info,investment_strategy,sector_weightings,top_holdings,total_assets,ytd_return,beta_3y,expense_ratio,inception_date
  symbol category                             family  \
0    SPY     NULL  SPDR State Street Global Advisors   
1    QQQ     NULL                            Invesco   
2    VOO     NULL                           Vanguard   

                             management_info investment_strategy  \
0  {'managerName': None, 'managerBio': None}                NULL   
1  {'managerName': None, 'managerBio': None}                NULL   
2  {'managerName': None, 'managerBio': None}                NULL   

  sector_weightings top_holdings total_assets ytd_return beta_3y  \
0              NULL         NULL         NULL       NULL    NULL   
1              NULL         NULL         NULL       NULL    NULL   
2              NULL         NULL         NULL       NULL    NULL   

  expense_ratio inception_date  
0          NULL           NULL 